In [1]:
## Basic rounding to 2dp
function rounding(x)
    return (floor(x*100))/100
end

rounding (generic function with 1 method)

In [2]:
# Changing to correct directory
cd("/home/simon/Documents/jobHunt/resumeWebsite/dataScience/sportsBetting/spreadsheetFiles")
# Uploading player stats
playerStats = readcsv("APTStats.csv")

106x143 Array{Any,2}:
 "Player"               "Rank 2014"  …   "2012 Ret BP P(Convert)"
 "Djokovic N."         1                0.46                     
 "Federer R."          2                0.42                     
 "Nadal R."            3                0.49                     
 "Wawrinka S."         4                0.43                     
 "Nishikori K."        5             …  0.39                     
 "Murray A."           6                0.41                     
 "Berdych T."          7                0.42                     
 "Raonic M."           8                0.39                     
 "Cilic M."            9                0.42                     
 "Ferrer D."          10             …  0.44                     
 "Dimitrov G."        11                0.36                     
 "Tsonga J.W."        12                0.42                     
 ⋮                                   ⋱                           
 "Lacko L."           94                0.35          

In [3]:
## Locating a specific data column
function dataSort(statName, data)
    for i = 1:size(data,2)
        if statName == data[1,i]
            return i
        end 
    end
end

# F.E. the column number for "Rank 2012"
dataSort("Clay P(Wc)-P(Wt)", playerStats)

29

In [4]:
# Find a specific player's data
function playerOdds(player, data)
    for i = 1:size(data,1)
        if data[i,1] == player
            return data[i,1:end]
    end end
    return"No Stats Available"
end

# F.E. the column number for "Rank 2012"
playerOdds("Raonic M.", playerStats)

1x143 Array{Any,2}:
 "Raonic M."  8  4440  2.49  11  2860  …  0.4693  0.33  317  0.3899  0.39

In [5]:
# Returns a players break point dominance ratio
# Key:
# bPRF: Break Point Return Faced, bPRC: Break Point Return Converted
# bPSR: Break Point Serve Faced,  bPSS: Break Point Serve Saved
function breakPointDomRatio(player,year,data)
    pOdds = playerOdds(player,data)
    bPSF = pOdds[dataSort(string(year," Ser BP P(Faced)"),data)]
    bPSS = pOdds[dataSort(string(year," Ser BP P(Saved)"),data)]
    bPRF = pOdds[dataSort(string(year," Ret BP P(Game)"),data)]
    bPRC = pOdds[dataSort(string(year," Ret BP P(Convert)"),data)]
    return (bPRF*bPRC)/bPSF*(1-bPRC)
end

# F.E. break point dominance ratio of Murray in 2012
breakPointDomRatio("Murray A.", 2012, playerStats)

0.36367245003090876

In [6]:
function ratioOdds(player1,player2, Open, year, data)
    # to find basic dom ratios for specific year
    # *** Turn to exp decaying rate ***
    domRatio = dataSort(string(year," Dom Ratio Games"),data)
    # to find surface variance for specific open
    if Open == "UK" || Open == "Queens" || Open == "Newport" || Open == "Hertogenbosch" ||
    Open == "Eastbourne"
        surfaceVar = dataSort("Grass P(Wg)-P(Wt)",data)
    elseif Open == "French" || Open == "Madrid" || Open == "MonteCarlo" ||
    Open == "Rome" || Open == "Umag" || Open == "Stuttgart" || Open == "Nice" ||
    Open == "Munich" || Open == "Kitzbuhel" || Open == "Houston" || Open == "Hamburg" ||
    Open == "Gstaad" || Open == "Charleston" || Open == "Casablanca" || 
    Open == "Buenosaires" || Open == "Bucharest" || Open == "Bogota" || 
    Open == "Bastad" || Open == "Baracelona" || Open == "Acapulco"
        surfaceVar = dataSort("Clay P(Wc)-P(Wt)",data)
    else
        surfaceVar = dataSort("Hard P(Wh)-P(Wt)",data)
    end
    # Now we have surface variance and dom ratios, we can begin to create
    # our arficial betting line
    p1Odds = playerOdds(player1, data)
    p2Odds = playerOdds(player2, data)
    # Firstly get specific player odds
    domP1 = p1Odds[domRatio]
    domP2 = p2Odds[domRatio]
    sVarP1 = p1Odds[surfaceVar]
    sVarP2 = p2Odds[surfaceVar]
    bPDRP1 = breakPointDomRatio(player1,year,data)
    bPDRP2 = breakPointDomRatio(player2,year,data)
    Tot1 = domP1*sVarP1*bPDRP1 
    Tot2 = domP2*sVarP2*bPDRP2
    if (Tot1 - Tot2) > 0
        return [1 (Tot1/Tot2)]
    else
        return [0 (Tot2/Tot1)]
    end
end

# F.E. the ratioOdds result of Ferrer playing Nadal at the Madrid Open in 2012 is
ratioOdds("Ferrer D.","Nadal R.","Madrid",2012,playerStats)

1x2 Array{Float64,2}:
 0.0  1.76777

In [21]:
# Creates an artificial betting line
function artLine(player1,player2,Open,year,data)
    if summary(playerOdds(player1,data)) != "ASCIIString" && 
        summary(playerOdds(player2,data)) != "ASCIIString"
        odds = ratioOdds(player1,player2,Open,year,data)
        if odds[1] == 1
            return [player1 1+1/odds[2]; player2 1+odds[2]]
        else
            return [player2 1+1/odds[2]; player1 1+odds[2]]
        end
    else
        return "No Betting Line Available"
    end
end

# F.E. the artificial of Ferrer playing Nadal at the Madrid Open in 2012 is
# ** Uses the stats from the year before, need to work an exponentially decay year system
artLine("Ferrer D.","Nadal R.","Madrid",2012,playerStats)

2x2 Array{Any,2}:
 "Nadal R."   1.56568
 "Ferrer D."  2.76777